In [13]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API
import time

In [14]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [15]:
# for the pie chart data from the admission json to the ages df pickle
# fname_pie = "admissions.json"

def json_plk_pie(filename):
    with open(filename) as fobjp:
        data_pie = json.load(fobjp)

    datalist_pie = data_pie["data"]

    age_range = []

    for i in datalist_pie:
        for k,v in i.items():
            if k == "admissions":
                for d in v:
                    for k2,v2 in d.items():
                        if k2 == "age":
                            if v2 in age_range:
                                continue
                            else:
                                age_range.append(v2)

    def min_age(agerange):
        agerange=agerange.replace('+','') # remove the + from 90+
        start=agerange.split('_')[0]
        return int(start)

    age_range.sort(key=min_age)

    age_df=pd.DataFrame(index=age_range, columns=['rate'])

    df = {}

    for i in age_range:
        df[i] = 0

    age_df=pd.DataFrame(index=age_range, columns=['Percentage of Admissions'])

    for i in datalist_pie:
            for k,v in i.items():
                if k == "admissions":
                    for d in v:
                        # print(d)
                        ageband = d["age"]
                        # age_df.loc[ageband] = d["value"]
                        age_df.loc[ageband] = d["rate"]*10

    age_df.to_pickle("agedf.pkl")
    
    return age_range



In [16]:
#  function for admissions pkl
def json_pkl(filename):
    """This function takes a filename, processes the data in the file to produce a pkl file"""
    with open (filename) as fobj:
        data = json.load(fobj)
        
    datalist = data["data"]
    
    names = [dictionary["name"] for dictionary in datalist]

    # get unique names in datalist
    unames = []

    for i in names:
        if i in unames:
            continue
        else:
            unames.append(i)

    # create dictionary with unique names and COVID 19 hospital case count
    final2 = {}
    cases2= 0  

    # set initial value of keys in dictionary to zero
    for place in unames:
        final2[place] = 0

    # calculate accumulated cases for names and update dictionary
    for i in datalist:
        for key,value in i.items():
            for place,case in final2.items():
                if value == place:
                    final2[place] = (case + i["cases"])

    # create dataframe with index names and column cases
    casesdf = pd.DataFrame(index=unames, columns=["Cases"])

    # update dataframe with total cases
    for k,v in final2.items():
        casesdf.loc[k] = v

    # create data pickle for cases
    casesdf.to_pickle("casesdf.pkl")

    # calculate total cases for all areas
    total = {}
    total["total"] = 0

    x = 0

    for k,v in final2.items():
        x += v

    totaldf = pd.DataFrame(index=total, columns=["Cases"])

    totaldf.loc["total"] = x
    
    # create data pickle for total cases
    totaldf.to_pickle("totaldf.pkl")

In [17]:
fname_bar = "hospitalcases.json"

fname_pie = "admissions.json"
pie_range = json_plk_pie(fname_pie)

json_pkl(fname_bar)
json_plk_pie(fname_pie)

pk1 = pd.read_pickle("casesdf.pkl")
pk2 = pd.read_pickle("totaldf.pkl")

pk1.loc["London"]=5000
pk1.loc["South East"]=5000
pk1.loc["East of England"]=5000
pk1.loc["Midlands"]=5000
pk1.loc["North East and Yorkshire"]=5000
pk1.loc["North West"]=5000
pk1.loc["South West"]=5000

pk2.loc["total"] = 5000

In [18]:
tog = wdg.ToggleButtons(
    options=['Areas', 'Total', 'Age'],
    description='COVID19 Charts:',
    disabled=False,
    button_style='info', #  ''
    # tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
)

def graph(name):
    global pk1, pk2, pie_range
    
    if name == "Areas":
        pk1.plot(kind="bar")
    elif name == "Total":
        pk2.plot(kind="bar")
    elif name == "Age":
        # pie_range = json_plk_pie("admissions.json")
        age_df=pd.read_pickle("agedf.pkl")
        agecols=wdg.SelectMultiple(
            options= pie_range, # options available
            value= pie_range, # initial value
            rows= 5, # rows of the selection box
            description='Age Range',
            disabled=False
        )
        def pie(names):
            age_df.loc[list(names)].plot.pie(subplots = True)
            plt.show()

        age_df_output = wdg.interactive_output(pie, {"names":agecols} )

        display(agecols, age_df_output)
        
        
        
output = wdg.interactive_output(graph,{'name': tog})

# display(tog, output)

In [19]:
def access_api(button):
    #     #button info
    apibutton.icon="check"
    apibutton.disabled=True
    
    fname_bar = "hospitalcases.json"
    fname_pie = "admissions.json"
    
    filters_bar = [
        "areaType=nhsRegion"
        ]

    structure_bar = {
        "name":"areaName",
        "cases":"hospitalCases"
        }

    api_bar = Cov19API(filters = filters_bar, structure = structure_bar)

    datajson_bar = api_bar.get_json()


    with open(fname_bar, "wt") as fobj:
        json.dump(datajson_bar, fobj)
        
    
    filters_pie = [
        "areaType=nation"
        ]

    structure_pie = {
        "admissions": "cumAdmissionsByAge"
        }

    api_pie = Cov19API(filters = filters_pie, structure = structure_pie)

    data_pie = api_pie.get_json()

    with open(fname_pie, "wt") as fobj:
        json.dump(data_pie, fobj)

    # wait for data to load from api
    time.sleep(15)
    # take data from file and datawrangle to produce the two pikles
    json_pkl(fname_bar)
    json_plk_pie(fname_pie)
    # then read the pikles and rest the graphs again
    
    # pk4 = pd.read_pickle("casesdf.pkl")
    # pk4.plot(kind="bar", color ="Green")
    
    # pk4 = pd.read_pickle("casesdf.pkl")
    # pk5 = pd.read_pickle("totaldf.pkl")
    # pk6 = pd.read_pickle("agedf.pkl")
    

#     def graph2(name):
#         global pie_range
#         pk4 = pd.read_pickle("casesdf.pkl")
#         pk5 = pd.read_pickle("totaldf.pkl")
#         pk6 = pd.read_pickle("agedf.pkl")

#         if name == "Areas":
#             pk4.plot(kind="bar", color ="Red")
#         elif name == "Total":
#             pk5.plot(kind="bar", color = "Red")
#         elif name == "Age":
#             # pie_range = json_plk_pie("admissions.json")
#             age_df=pk6
#             agecols=wdg.SelectMultiple(
#                 options= pie_range, # options available
#                 value= pie_range, # initial value
#                 rows= 5, # rows of the selection box
#                 description='Age Range',
#                 disabled=False
#             )
#             def pie(names):
#                 age_df.loc[list(names)].plot.pie(subplots = True)
#                 plt.show()

#             age_df_output = wdg.interactive_output(pie, {"names":agecols} )

#             display(agecols, age_df_output)
            
    
#     output_new = wdg.interactive_output(graph2,{'name': tog})
            
#     # return output_new
#     display(output_new)

# see the doc for the parameters    
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)


apibutton.on_click(access_api)

display(tog, output, apibutton)

# x = apibutton.on_click(access_api)
# display(x)

ToggleButtons(button_style='info', description='COVID19 Charts:', options=('Areas', 'Total', 'Age'), value='Ar…

Output()

Button(button_style='success', description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Clic…

In [20]:
def access_api(button):
    global pk1, pk2
    #     #button info
    apibutton.icon="check"
    apibutton.disabled=True
    
    fname_bar = "hospitalcases.json"
    fname_pie = "admissions.json"
    
    filters_bar = [
        "areaType=nhsRegion"
        ]

    structure_bar = {
        "name":"areaName",
        "cases":"hospitalCases"
        }

    api_bar = Cov19API(filters = filters_bar, structure = structure_bar)

    datajson_bar = api_bar.get_json()


    with open(fname_bar, "wt") as fobj:
        json.dump(datajson_bar, fobj)
        
    
    filters_pie = [
        "areaType=nation"
        ]

    structure_pie = {
        "admissions": "cumAdmissionsByAge"
        }

    api_pie = Cov19API(filters = filters_pie, structure = structure_pie)

    data_pie = api_pie.get_json()

    with open(fname_pie, "wt") as fobj:
        json.dump(data_pie, fobj)

    # wait for data to load from api
    time.sleep(15)
    # take data from file and datawrangle to produce the two pikles
    json_pkl(fname_bar)
    json_plk_pie(fname_pie)
    # then read the pikles and rest the graphs again
    pk1 = pd.read_pickle("casesdf.pkl")
    pk2 = pd.read_pickle("totaldf.pkl")
    
    # pk4 = pd.read_pickle("casesdf.pkl")
    # pk4.plot(kind="bar", color ="Green")
    
    # pk4 = pd.read_pickle("casesdf.pkl")
    # pk5 = pd.read_pickle("totaldf.pkl")
    # pk6 = pd.read_pickle("agedf.pkl")
    

#     def graph2(name):
#         global pie_range, pk
#         pk4 = pd.read_pickle("casesdf.pkl")
#         pk5 = pd.read_pickle("totaldf.pkl")
#         pk6 = pd.read_pickle("agedf.pkl")

#         if name == "Areas":
#             pk4.plot(kind="bar", color ="Red")
#         elif name == "Total":
#             pk5.plot(kind="bar", color = "Red")
#         elif name == "Age":
#             # pie_range = json_plk_pie("admissions.json")
#             age_df=pk6
#             agecols=wdg.SelectMultiple(
#                 options= pie_range, # options available
#                 value= pie_range, # initial value
#                 rows= 5, # rows of the selection box
#                 description='Age Range',
#                 disabled=False
#             )
#             def pie(names):
#                 age_df.loc[list(names)].plot.pie(subplots = True)
#                 plt.show()

#             age_df_output = wdg.interactive_output(pie, {"names":agecols} )

#             display(agecols, age_df_output)
            
    
    output_new = wdg.interactive_output(graph2,{'name': tog})
            
    return output_new
    # display(output_new)

# see the doc for the parameters    
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)


apibutton.on_click(access_api)
# apibutton.on_click(output_new)

display(tog, output, apibutton)

# x = apibutton.on_click(access_api)
# display(x)

ToggleButtons(button_style='info', description='COVID19 Charts:', options=('Areas', 'Total', 'Age'), value='Ar…

Output()

Button(button_style='success', description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Clic…

In [21]:
pk1

,Cases
London,5000
South East,5000
East of England,5000
Midlands,5000
North East and Yorkshire,5000
North West,5000
South West,5000


In [23]:
# def graph2(name):
#         global pie_range
#         pk4 = pd.read_pickle("casesdf.pkl")
#         pk5 = pd.read_pickle("totaldf.pkl")
#         pk6 = pd.read_pickle("agedf.pkl")

#         if name == "Areas":
#             pk4.plot(kind="bar", color ="Red")
#         elif name == "Total":
#             pk5.plot(kind="bar", color = "Red")
#         elif name == "Age":
#             # pie_range = json_plk_pie("admissions.json")
#             age_df=pk6
#             agecols=wdg.SelectMultiple(
#                 options= pie_range, # options available
#                 value= pie_range, # initial value
#                 rows= 5, # rows of the selection box
#                 description='Age Range',
#                 disabled=False
#             )
#             def pie(names):
#                 age_df.loc[list(names)].plot.pie(subplots = True)
#                 plt.show()

#             age_df_output = wdg.interactive_output(pie, {"names":agecols} )

#             display(agecols, age_df_output)
